# ДЗ к «Функции и работа с данными»

In [1]:
import pandas as pd

### Задание 1**   
Напишите функцию, которая классифицирует фильмы из материалов занятия по следующим правилам:   
- оценка 2 и меньше - низкий рейтинг   
- оценка 4 и меньше - средний рейтинг   
- оценка 4.5 и 5 - высокий рейтинг   

Результат классификации запишите в столбец class

In [2]:
df_ratings = pd.read_csv('ratings.csv')

In [3]:
def class_movie(class_):
    if class_ <= 2:
        return 'Низкий рейтинг'
    elif class_ <= 4:
        return 'Средний рейтинг'
    else:
        return 'Высокий рейтинг'

df_ratings['class'] = df_ratings.rating.apply(class_movie)

In [4]:
qry = "rating < 3"

res = df_ratings.query(qry)

res.head()

,userId,movieId,rating,timestamp,class
0,1,31,2.5,1260759144,Средний рейтинг
3,1,1129,2.0,1260759185,Низкий рейтинг
5,1,1263,2.0,1260759151,Низкий рейтинг
6,1,1287,2.0,1260759187,Низкий рейтинг
7,1,1293,2.0,1260759148,Низкий рейтинг


### Задание 2   
Используем файл keywords.csv.   

Необходимо написать гео-классификатор, который каждой строке сможет выставить географическую принадлежность определенному региону. Т. е. если поисковый запрос содержит название города региона, то в столбце ‘region’ пишется название этого региона. Если поисковый запрос не содержит названия города, то ставим ‘undefined’.

In [7]:
df_geo = pd.read_csv('keywords.csv')
# df_geo.head()

In [8]:
geo_data = {'Центр': ['москва', 'тула', 'ярославль'],
            'Северо-Запад': ['петербург', 'псков', 'мурманск'],
            'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']
           }

In [9]:
geo_data

{'Центр': ['москва', 'тула', 'ярославль'],
 'Северо-Запад': ['петербург', 'псков', 'мурманск'],
 'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']}

In [10]:
def name_region(keyword_list):
    """
    Пробегаемся по словарю -> затем пробегаемся по значением словаря
    Если значение словаря попадает в параметр функции, то выводим ключ словаря
    """
    for keys, values in geo_data.items():
        for word in values:
            if word in keyword_list:
                return keys
    return 'undefined'

df_geo['region'] = df_geo.keyword.apply(name_region)

df_geo.groupby(['region']).count()

,keyword,shows
region,,
undefined,99253,99253
Дальний Восток,94,94
Северо-Запад,266,266
Центр,387,387


In [11]:
"""
Сам не додумался, но нашел альтернативный вариант с функцией any():
Более лаконичный вариант
"""
def name_region(keyword_list):
    for keys, values in geo_data.items():
        if any (word in keyword_list for word in values):
            return keys
    return 'undefined'

df_geo['region'] = df_geo.keyword.apply(name_region)

### Задание 3 (бонусное)

Есть мнение, что раньше снимали настоящее кино, не то что сейчас. Ваша задача — проверить это утверждение, используя файлы с рейтингами фильмов из прошлого домашнего занятия: файл movies.csv и ratings.csv из базы. Нужно проверить, верно ли, что с ростом года выпуска фильма его средний рейтинг становится ниже.

Вы не будете затрагивать субьективные факторы выставления этих рейтингов, а пройдётесь по алгоритму:

1.   В переменную years запишите список из всех годов с 1950 по 2010 года.
2.   Напишите функцию production_year, которая каждой строке из названия фильма выставляет год выпуска. Не все названия фильмов содержат год выпуска в одинаковом формате, поэтому используйте алгоритм:
   *   для каждой строки пройдите по всем годам списка years;
   *   если номер года присутствует в названии фильма, то функция возвращает этот год, как год выпуска;
   *   если ни один из номеров года списка years не встретился в названии фильма, то возвращается 1900 год.
3.   Запишите год выпуска фильма по алгоритму пункта 2 в новый столбец ‘year’.
4.   Посчитайте средний рейтинг всех фильмов для каждого значения столбца ‘year’ и отсортируйте результат по убыванию рейтинга.

In [ ]:
df_movies = pd.read_csv('movies.csv')
df_ratings = pd.read_csv('ratings.csv')

In [ ]:
"""
Как я намучался с этой строкой :-(
"""
list_year = [str(i) for i in range(1950,2010)]

def production_year(row):
    for year in list_year:
        if year in row['title']:
            return year
    return '1990'

df_movies['year'] = df_movies.apply(production_year, axis=1)

"""
Присоединяем к таблице фильмы все значение из таблицы рейтингов
"""
movies = df_movies.merge(df_ratings, left_on='movieId', right_on='movieId', how='left')

"""
На вывод определяем два столбца с группировкой по годам и сортировкой по убыванию среднего значения по рейтингк
"""
movies[['year', 'rating']].groupby('year').mean('rating').sort_values('rating', ascending=False)

In [ ]:
"""
Переменная годя является текстом, к сожалению.
Для проверки корректности, выводим наш DF по фильтру вхождения символов в значение столбца year 
"""
df_movies[df_movies['year'].str.contains('2009')]

"""
либо фильтруем с помощью метода loc[].
loc[] используется для доступа к значениям из столбцов DataFrame

df = df_movies.loc[df_movies['year'] == '2009']
"""

# ДЗ к «Продвинутый pandas»

In [ ]:
import pandas as pd
import time
import numpy as np

### Задание 1
Для датафрейма log из материалов занятия создайте столбец source_type по правилам:
*   если источник traffic_source равен Yandex или Google, то в source_type ставится organic;
* для источников paid и email из России ставим ad;
*   для источников paid и email не из России ставим other;
* все остальные варианты берём из traffic_source без изменений.

In [ ]:
df_log = pd.read_csv('visit_log.csv', sep=';')
df_log.head()

,timestamp,visit_id,url,region,user_id,traffic_source
0,1549980692,e3b0c44298,https://host.ru/3c19b4ef7371864fa3,Russia,b1613cc09f,yandex
1,1549980704,6e340b9cff,https://host.ru/c8d9213a31839f9a3a,Russia,4c3ec14bee,direct
2,1549980715,96a296d224,https://host.ru/b8b58337d272ee7b15,Russia,a8c40697fb,yandex
3,1549980725,709e80c884,https://host.ru/b8b58337d272ee7b15,Russia,521ac1d6a0,yandex
4,1549980736,df3f619804,https://host.ru/b8b58337d272ee7b15,Russia,d7323c571c,yandex


In [ ]:
df_log.shape

In [ ]:
df_log.loc[(df_log.traffic_source == 'yandex') | (df_log.traffic_source == 'google'), 'source_type'] = 'organic'
df_log.loc[((df_log.traffic_source == 'paid') | (df_log.traffic_source == 'email')) & (df_log.region=='Russia'), 'source_type'] = 'ad'
df_log.loc[((df_log.traffic_source == 'paid') | (df_log.traffic_source == 'email')) & (df_log.region!='Russia'), 'source_type'] = 'other'
df_log.source_type = df_log.source_type.fillna(df_log.source_type)

In [ ]:
"""
короткий вариант объединения условия "И"
"""
# df_log.loc[df_log.traffic_source.isin(['yandex', 'google']), 'traffic_type'] = 'organic'

In [ ]:
qry = "source_type not in ['organic', 'ad', 'other']"
df_log.query(qry).head()

### Задание 2
В файле URLs.txt содержатся URL страниц новостного сайта. Вам нужно отфильтровать его по адресам страниц с текстами новостей. Известно, что шаблон страницы новостей имеет внутри URL конструкцию: /, затем 8 цифр, затем 
дефис.   
Выполните действия:
1. Прочитайте содержимое файла с датафрейм.
2. Отфильтруйте страницы с текстом новостей, используя метод str.contains и регулярное выражение в соответствие с заданным шаблоном.

In [ ]:
df_urls = pd.read_csv('URLs.txt')
df_urls.head()

,url
0,/world/
1,/latest/
2,/?updated=top
3,/politics/36188461-s-marta-zhizn-rossiyan-susc...
4,/world/36007585-tramp-pridumal-kak-reshit-ukra...


In [ ]:
"""
\\d означает любую цифру
"""
df_urls[df_urls.url.str.contains('/\\d\\d\\d\\d\\d\\d\\d\\d-')].head()

,url
3,/politics/36188461-s-marta-zhizn-rossiyan-susc...
4,/world/36007585-tramp-pridumal-kak-reshit-ukra...
5,/science/36157853-nasa-sobiraet-ekstrennuyu-pr...
6,/video/36001498-poyavilis-pervye-podrobnosti-g...
7,/world/36007585-tramp-pridumal-kak-reshit-ukra...


### Задание 3
Используйте файл с оценками фильмов ml-latest-small/ratings.csv. Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок. Под временем жизни понимается разница между максимальным и минимальным значениями столбца timestamp для данного значения userId.

In [ ]:
df_ratings = pd.read_csv('ratings.csv')
df_ratings['date'] = pd.to_datetime(df_ratings['timestamp'], unit='s')
df_ratings.head()

,userId,movieId,rating,timestamp,date
0,1,31,2.5,1260759144,2009-12-14 02:52:24
1,1,1029,3.0,1260759179,2009-12-14 02:52:59
2,1,1061,3.0,1260759182,2009-12-14 02:53:02
3,1,1129,2.0,1260759185,2009-12-14 02:53:05
4,1,1172,4.0,1260759205,2009-12-14 02:53:25


Я запарился танцевать с бубнами, чтобы отфильтровать по критерию более 100 оценок.   
Можно было сделать несколько группировок и потом соединить их через megre, но такой вариант мне не понравился.  

Нашел вариант:   

```
# uid_count_lt = tmp.groupby('userId')['timestamp']\
        .agg({'count',  lambda x: max(x) - min(x)}).reset_index()\
        .rename(columns={'':'lt'})
uid_count_lt[uid_count_lt['count'] > 100]['lt'].mean()
```
но так и не смог в него внести фильтр по оценкам, чтобы сразу считал (по аналогии с having()), а полность брать чужой код не захотел.   

Сделал нужную выборку через SQL (на что потратил вместо 5 часов 15 минут)...

In [ ]:
import sqlite3 as sl
con = sl.connect('my-base.db')
# df_ratings.to_sql('test', con)

In [ ]:
# %time
df_new = pd.read_sql('''
    SELECT userId
        , count(rating) as 'count_rating'
        , min(date) as 'start'
        , max(date) as 'end'
        , max(timestamp)-min(timestamp) as 'time_life'
    FROM test
    group by userId
    having count(rating) > 100
''', con)
df_new.head()

,userId,count_rating,start,end,time_life
0,4,204,2000-02-05 19:25:14,2000-02-08 03:57:54,203560
1,8,116,2006-07-31 23:42:20,2006-08-01 23:22:07,85187
2,15,1700,2001-08-16 04:47:19,2016-07-24 03:25:35,471393496
3,17,363,2005-09-23 09:43:07,2005-09-23 11:57:20,8053
4,19,423,1997-02-06 00:48:11,1997-02-06 02:16:13,5282


In [ ]:
f = df_new['time_life'].mean().round(2)
print(f'Среднее время жизни всех пользователей, давших более 100 оценок:\nчч:мм:сс\n{time.strftime("%H:%M:%S", time.gmtime(f))}')

Среднее время жизни всех пользователей, давших более 100 оценок:
чч:мм:сс
21:28:27


In [ ]:
"""
В задании 3 можно сделать необходимые вычисления в одной группировке с помощью метода agg:
df_ratings.groupby('userId').agg({'timestamp': ['min', 'max', 'count']})
Затем останется только отфильтровать по числу оценок и посчитать среднюю разницу min и max.
"""

### Задание 4
Дана статистика услуг перевозок клиентов компании по типам (см. файл “Python_13_join.ipynb” в разделе «Материалы для лекции “Продвинутый pandas”» ---- Ноутбуки к лекции «Продвинутый pandas»).   
Нужно сформировать две таблицы:
* таблицу с тремя типами выручки для каждого client_id без указания адреса клиента;
* аналогичную таблицу по типам выручки с указанием адреса клиента.

In [ ]:
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)

auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)

air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)

client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)

In [ ]:
from functools import reduce

In [ ]:
reduce(lambda a, b: a.merge(b, how='left', on='client_id'), [client_base, rzd, auto, air]).fillna(0).drop(columns='address')

,client_id,rzd_revenue,auto_revenue,air_revenue
0,111,1093.0,0.0,0.0
1,112,2810.0,0.0,0.0
2,113,10283.0,57483.0,0.0
3,114,5774.0,83.0,0.0
4,115,981.0,912.0,81.0
5,116,0.0,4834.0,4.0
6,117,0.0,98.0,13.0
7,118,0.0,0.0,173.0
